# MLFlow

It can be very hard to keep track of all of the different machine learning experiments on ran. Luckily we have mlflow too keep track of the configurations and the results accociated with it.

In [2]:
import mlflow
import pandas as pd
df = pd.read_csv('train.csv')
df = df.fillna(0)
features = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
target_column = ["Survived"]

In [8]:
from typing import Dict
import operator

def get_family_id(row: pd.Series, family_id_map: Dict[str, int]) -> int:
    # Find the last name by splitting on a comma
    last_name = row["Name"].split(",")[0]
    # Create the family id
    family_id = "{0}{1}".format(last_name, row["FamilySize"])
    # Look up the id in the mapping
    if family_id not in family_id_map:
        if len(family_id_map) == 0:
            current_id = 1
        else:
            # Get the maximum id from the mapping and add one to it if we don't have an id
            current_id = (max(family_id_map.items(), key=operator.itemgetter(1))[1] + 1)
        family_id_map[family_id] = current_id
    return family_id_map[family_id]

famile_id_map = {}
df["FamilySize"] = df["SibSp"] + df["Parch"]
df["FamilyId"] = df.apply(get_family_id, args=(famile_id_map,), axis=1)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,FamilyId
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,3
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,4
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,698
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,0,699
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,3,633
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,0,700


In [44]:
from mlflow.entities import Run, Experiment
import json
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import mlflow.sklearn

mlflow.set_experiment("Titanic")

df = df.fillna(0)

xTrain, xTest, yTrain, yTest = train_test_split(
    df[features],
    df["Survived"],
    stratify = df["Survived"],
    random_state = 42
)


with mlflow.start_run() as run:
    max_depth = 3
    max_features = 4
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("max_features", max_features)
    decision_tree = DecisionTreeClassifier(
        max_depth=max_depth,
        max_features=max_features
    )
    decision_tree.fit(df[features], df[target_column])
    
    train_accuracy = decision_tree.score(xTrain, yTrain)
    mlflow.log_metric("train_accuracy", train_accuracy)
    
    test_accuracy = decision_tree.score(xTest, yTest)
    mlflow.log_metric("test_accuracy", test_accuracy)
    with open("family_id_map.json", "w") as f:
        json.dump(famile_id_map, f)
    mlflow.log_artifact("family_id_map.json")
    mlflow.sklearn.log_model(decision_tree, "model")



In [45]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
client.list_experiments()

[<Experiment: artifact_location='file:///Users/jendrikjordening/go/src/gitlab.com/nooxit/ml_workshop_qcon/Titanic/mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='file:///Users/jendrikjordening/go/src/gitlab.com/nooxit/ml_workshop_qcon/Titanic/mlruns/1', experiment_id='1', lifecycle_stage='active', name='Titanic', tags={}>]

In [46]:
client.list_run_infos("1")

[<RunInfo: artifact_uri='file:///Users/jendrikjordening/go/src/gitlab.com/nooxit/ml_workshop_qcon/Titanic/mlruns/1/fbed2be51a344f698c34cc9244f13dcc/artifacts', end_time=1583324567545, experiment_id='1', lifecycle_stage='active', run_id='fbed2be51a344f698c34cc9244f13dcc', run_uuid='fbed2be51a344f698c34cc9244f13dcc', start_time=1583324567493, status='FINISHED', user_id='jendrikjordening'>,
 <RunInfo: artifact_uri='file:///Users/jendrikjordening/go/src/gitlab.com/nooxit/ml_workshop_qcon/Titanic/mlruns/1/c9c96c5221b24646b39fe3db883b24eb/artifacts', end_time=1583324556131, experiment_id='1', lifecycle_stage='active', run_id='c9c96c5221b24646b39fe3db883b24eb', run_uuid='c9c96c5221b24646b39fe3db883b24eb', start_time=1583324556094, status='FAILED', user_id='jendrikjordening'>]

In [49]:
! ls mlruns/1/fbed2be51a344f698c34cc9244f13dcc/artifacts/model

MLmodel    conda.yaml model.pkl
